In [123]:
import requests
from requests.api import request
import time
import json
import re
import random
import os
import pickle

previousRequestsBackupPath = "./previousRequests/"

def get_valid_filename(name):
    s = str(name).strip().replace(" ", "_")
    s = re.sub(r"(?u)[^-\w.]", "", s)
    if s in {"", ".", ".."}:
        return "ErrorFileName" + str(random.randrange(100000))
    return s

def get_valid_path(path):
    s = str(path).strip().replace(" ", "_")
    s = re.sub(r"(?u)[^-\w./]", "", s)
    if s in {"", ".", ".."}:
        return "ErrorFileName" + str(random.randrange(100000))
    return s

def dumpAPIRequest(fileName, data, path="./previousRequests/"):
    fullFileName = get_valid_path(path) + get_valid_filename(fileName) + ".bin"
    os.makedirs(os.path.dirname(fullFileName), exist_ok=True)
    with open(fullFileName, "wb") as outputFile:
        pickle.dump(data, outputFile)
    return fullFileName

def loadAPIRequest(fileName):
    fullFileName = previousRequestsBackupPath + get_valid_filename(fileName) +".bin"
    if not os.path.exists(fullFileName):
        return {}
    with open(fullFileName, "rb") as inputFile:
        data = pickle.load(inputFile)
    return data["result"]

def checkForAPIRequest(url):
    return os.path.exists(previousRequestsBackupPath + get_valid_filename(url) +".bin")

def dumpJSON(fileName, data, path="./"):
    if len(data) == 0:
        return ""
    fullFileName = get_valid_path(path) + get_valid_filename(fileName) + ".json"
    os.makedirs(os.path.dirname(fullFileName), exist_ok=True)
    with open(fullFileName, "w") as outputFile:
        json.dump(data, outputFile, indent = 6)
    return fullFileName

def loadJSON(fullFileName):
    if not os.path.exists(fullFileName):
        return {}
    with open(fullFileName, "r") as inputFile:
        data = json.load(inputFile)
    return data

def checkForJSON(fileName, path):
    fullFileName = get_valid_path(path) + get_valid_filename(fileName) + ".json"
    return loadJSON(fullFileName) if os.path.exists(fullFileName) else {}


In [124]:
lastRequestTime = 0
personalAccessToken = "116X4PZSFPNP2AHJGQK3S85REGS81HQFN2"

def apiRequest(url):
    if checkForAPIRequest(url):
        return loadAPIRequest(url)
    
    global lastRequestTime
    while(time.time() < lastRequestTime+0.2):
        pass
    lastRequestTime = time.time()

    response = requests.get(url + "&apikey=" + personalAccessToken)
    data = response.json() if response.ok else {}

    dumpAPIRequest(url, data)

    return data["result"]


In [136]:
apiString = "https://api.etherscan.io/api"

def getAllResults(url, backupName=None):
    endBlock = "99999999"
    startBlock = "0"
    maxAmount = 10000
    url = url + "&endblock=" + endBlock + "&startblock="
    singleList = apiRequest(url + str(startBlock))
    all = singleList
    dumpMultiplier = 1
    while(len(singleList) >= maxAmount):
        if(len(all) > 100000 * dumpMultiplier):
            dumpJSON(backupName if backupName is not None else url, all, "./data/Backups/")
            dumpMultiplier += 1
        lastElement = singleList[-1]
        startBlock = lastElement["blockNumber"]
        offset = len(singleList) - next((i for i, item in enumerate(singleList) if item["blockNumber"] == startBlock))
        singleList = apiRequest(url + str(startBlock))
        truncatedSingleList = singleList[offset:]
        if(len(truncatedSingleList) > 0):
            all += truncatedSingleList
    return all

moduleString = "module="
accountModuleString = moduleString + "account"
actionString = "action="
tokenTxActionString = actionString + "tokentx"
contractAdressString = "contractaddress="
def getTokenTransactions(address):
    data = getAllResults(apiString + "?" + accountModuleString + "&" + tokenTxActionString + "&" + contractAdressString + address, "tokens-" + address)
    dumpJSON(address, data, "./data/tokens/")
    return data

tokenNftTxActionString = actionString + "tokennfttx"
def getNftTransactions(address):
    data = getAllResults(apiString + "?" + accountModuleString + "&" + tokenNftTxActionString + "&" + contractAdressString + address, "nfts-" + address)
    dumpJSON(address, data, "./data/nfts/")
    return data

token1155txActionString = actionString + "token1155tx"
def getMultiTokenTransactions(address):
    data = getAllResults(apiString + "?" + accountModuleString + "&" + token1155txActionString + "&" + contractAdressString + address, "MultiTokens-" + address)
    dumpJSON(address, data, "./data/MultiTokens/")
    return data

txlistActionString = actionString + "txlist"
adressString = "address="
def getAddressTransactions(address):
    data = getAllResults(apiString + "?" + accountModuleString + "&" + txlistActionString + "&" + adressString + address, "normal-" + address)
    dumpJSON(address, data, "./data/normal/")
    return data

txlistInternalActionString = actionString + "txlistinternal"
def getAddressInternalTransactions(address):
    data = getAllResults(apiString + "?" + accountModuleString + "&" + txlistInternalActionString + "&" + adressString + address, "internal-" + address)
    dumpJSON(address, data, "./data/internal/")
    return data

def getAddressData(address):
    data = getAllResults(apiString + "?" + accountModuleString + "&" + txlistActionString + "&" + adressString + address, "addresses-" + address)
    dumpJSON(address, data, "./data/addresses/")
    return data


In [137]:
addresses = ["0x18dd4e0eb8699eA4FeE238dE41ECfb95e32272f8", "0xe19ae8F9B36Ca43D12741288D0e311396140DF6F"]

for address in addresses:
    transactionData = getAddressData(address)
    for transaction in transactionData:
        if transaction["methodId"] == "0x60806040":
            getTokenTransactions(transaction["contractAddress"])
            getNftTransactions(transaction["contractAddress"])
            getMultiTokenTransactions(transaction["contractAddress"])
            getAddressTransactions(transaction["contractAddress"])
            getAddressInternalTransactions(transaction["contractAddress"])
